## 설정

In [15]:
import shutil
import datetime
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')

from chart_portal import *
from table_portal import *
from docx.text.run import *
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [2]:
# DRM 방지(default.docx)
def findfile(name, path):
    for dirpath, dirname, filename in os.walk(path):
        if name in filename:
            return os.path.join(dirpath, name)

file_path=os.path.abspath("")
srcpath = findfile("default.docx.org", file_path)
dir, file = os.path.split(srcpath)
shutil.copy2(srcpath, dir+"\default.docx")

'C:\\Beomjun\\venv\\Lib\\site-packages\\docx\\templates\\default.docx'

In [3]:
from document_portal import *
# report version
rtype = '월간' # 연간, 반기

In [4]:
# 메타 정보

today = datetime.date.today()

y = today.year
m = today.month
d = today.day
w = today.weekday()

v = '1'

# if region == 'All':
#     regex = '(KR|CN|NA|EU|SG|RU|IN|Global|All)'
# else:
#     regex = region

In [5]:
# Document 기본 폰트
style = document.styles['Normal']
# style.font.name = '맑은고딕'
style.font.name = 'Calibri'
style.font.size = Pt(12)
# style._element.rPr.rFonts.set(qn('w:eastAsia'), '맑은고딕')
style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Calibri')

In [6]:
# Chart 기본 폰트
alt.themes.register('맑은고딕', hanfont)
alt.themes.enable('맑은고딕')

ThemeRegistry.enable('맑은고딕')

## VM 집계

In [58]:
import time
from datetime import datetime, timedelta
from novaclient import client
import pandas as pd

import os
from novaclient import client
from keystoneauth1 import session
from keystoneauth1.identity import v3
from odf.opendocument import OpenDocumentSpreadsheet
from odf.table import Table, TableRow, TableCell
from odf.text import P
from keystoneclient.v3 import client as keystone_client

In [59]:
def create_ods_file(file_path, header, data_rows):
    print(" ODS 파일 생성 시작...")

    # Windows 경로 맞춤 설정
    file_path = file_path.replace("/", "\\")

    #  만약 동일한 파일이 있으면 삭제
    if os.path.exists(file_path):
        os.remove(file_path)

    #  ODS Document 생성
    doc = OpenDocumentSpreadsheet()
    table = Table(name="VM_Data")

    #  Header 추가
    print(f"📝 Header 추가 중: {header}")
    header_row = TableRow()
    for column_name in header:
        cell = TableCell()
        # 👉 value-type을 string으로 명시
        cell.setAttribute("valuetype", "string")
        cell.addElement(P(text=str(column_name)))
        header_row.addElement(cell)
    table.addElement(header_row)

    #  Data 추가
    print(f"📝 총 {len(data_rows)}개의 데이터가 추가됩니다.")
    for idx, row in enumerate(data_rows):
        # print(f"➡️ [{idx+1}/{len(data_rows)}] {row}")
        row_element = TableRow()
        for cell_data in row:
            if cell_data is None:
                cell_data = ""
            cell = TableCell()
            cell.setAttribute("valuetype", "string")
            cell.addElement(P(text=str(cell_data)))
            row_element.addElement(cell)
        table.addElement(row_element)

    # Table을 Spreadsheet에 추가
    print(f"💾 Spreadsheet에 Table 추가")
    doc.spreadsheet.addElement(table)

    #  파일 직접 쓰기
    print(f"💾 ODS 파일 쓰기 시작... {file_path}")
    try:
        with open(file_path, "wb") as f:
            doc.write(f)      # 명시적으로 write 수행
            f.flush()         # 버퍼에 남아있는 내용 비우기
        print(f"✅ ODS 파일 생성 완료: {file_path}")
    except Exception as e:
        print(f"❌ 파일 저장 실패: {e}")



# 파일명 생성 함수
def get_file_and_topic_names(current_date):
    file_name = f'C:\\Beomjun\\csv\\VM\\vm_info_{current_date}.ods'
    return file_name

In [60]:
def nova_extract(project_name):
    current_date = datetime.now().strftime("%Y%m%d")

    auth = v3.Password(
        auth_url='http://cloud-control-vip.eu-central.openstack.h53:5000/v3',
        username='admin',
        password='TldhTl1!',
        user_domain_name='Default',
        project_name='admin',
        project_domain_name='Default'
    )

    sess = session.Session(auth=auth)
    nova = client.Client('2.1', session=sess)

    # 모든 프로젝트 VM 가져오기
    vms = nova.servers.list(search_opts={'all_tenants': 1})

    # 프로젝트 리스트
    keystone = keystone_client.Client(session=sess)
    projects = keystone.projects.list()
    project_map = {p.id: p.name for p in projects}  # id → name 맵 생성
    
    ods_file_path = get_file_and_topic_names(current_date)
    print("ODS 파일명:", ods_file_path)
    
    ods_header = ["project_id", "id", "name", "status", "flavor", "address", "availability_zone", "hostname", "created_at", "updated_at"]
    data_rows = []

    for vm in vms:
        project_name = project_map.get(vm._info.get('tenant_id'), "unknown")
        
        addresses_info = ""
        for key, values in vm.addresses.items():
            for addr_info in values:
                addresses_info += f'{key}:{addr_info["addr"]}\n'
        addresses_info = addresses_info.strip()

        flavor_id = vm.flavor['id']
        flavor = nova.flavors.get(flavor_id)

        try:
            flavor = nova.flavors.get(flavor_id)
            flavor_name = flavor.name
        except Exception as e:
            flavor_name = ""

        vm_data = [
            project_name, vm.id, vm.name, vm.status, flavor_name, addresses_info,
            vm._info['OS-EXT-AZ:availability_zone'], vm._info['OS-EXT-SRV-ATTR:host'],
            vm.created, vm.updated
        ]
        data_rows.append(vm_data)
        # print("data!", vm_data)

    
    # print("data_rows!", data_rows)
    # ODS 파일 생성
    create_ods_file(ods_file_path, ods_header, data_rows)
    return ods_file_path

In [62]:
# 오래된 파일 정리
def cleanup(tenants):
    time.sleep(1)
    current_date = datetime.now().strftime("%Y%m%d")
    retention_period = 5
    delete_date = (datetime.now() - timedelta(days=retention_period)).strftime("%Y%m%d")
    host_dir = 'C://Beomjun//csv//'
    file_list = os.listdir(host_dir)

    for tenant in tenants:
        for file_name in file_list:
            if file_name.startswith(f'vm_eu2_{tenant}_') and file_name <= f'vm_eu2_{tenant}_{delete_date}.ods':
                file_path = os.path.join(host_dir, file_name)
                os.remove(file_path)
                print("file remove success!")

project_names = ['admin']
tenants = ['adminrc']

# 각 프로젝트에 대해 함수 실행
for project_name in project_names:
    print("This project is", project_name)
    ods_file_path = nova_extract(project_name)
    # ods_file_path = nova_extract_admin(project_name)

# cleanup(tenants)

This project is admin
ODS 파일명: C:\Beomjun\csv\VM\vm_info_20250527.ods
 ODS 파일 생성 시작...
📝 Header 추가 중: ['project_id', 'id', 'name', 'status', 'flavor', 'address', 'availability_zone', 'hostname', 'created_at', 'updated_at']
📝 총 2038개의 데이터가 추가됩니다.
💾 Spreadsheet에 Table 추가
💾 ODS 파일 쓰기 시작... C:\Beomjun\csv\VM\vm_info_20250527.ods
✅ ODS 파일 생성 완료: C:\Beomjun\csv\VM\vm_info_20250527.ods


In [50]:
# 날짜 설정
current_date = datetime.now().strftime("%Y%m%d")
yesterday_date = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")

print("today:", current_date)
print("yesterday:", yesterday_date)

# 경로 설정
today_path = f"C:\\Beomjun\\csv\\VM\\vm_info_{current_date}.ods"
yesterday_path = f"C:\\Beomjun\\csv\\VM\\vm_info_{yesterday_date}.ods"

# file_path=os.path.abspath("")
# srcpath = findfile(f"eu_adminrc_{yesterday_date}.csv.org", file_path)
# dir, file = os.path.split(srcpath)
# shutil.copy2(srcpath, dir+f"\eu_adminrc_{yesterday_date}.csv")
    
# CSV 로드
df_today = pd.read_excel(today_path, engine='odf')
df_yesterday = pd.read_excel(yesterday_path, engine='odf')

today: 20250527
yesterday: 20250526


In [51]:
# ID 기준으로 차집합 연산
today_ids = set(df_today['id'])
yesterday_ids = set(df_yesterday['id'])

# 추가된 VM: 오늘에는 있고 어제는 없던 것
added_ids = today_ids - yesterday_ids
df_added = df_today[df_today['id'].isin(added_ids)]

# 삭제된 VM: 어제는 있었는데 오늘은 없는 것
deleted_ids = yesterday_ids - today_ids
df_deleted = df_yesterday[df_yesterday['id'].isin(deleted_ids)]

# 결과 출력
print("\n✅ 오늘 추가된 VM:")
print(df_added)

print("\n❌ 오늘 삭제된 VM:")
print(df_deleted)


✅ 오늘 추가된 VM:
                                project_id  \
0     d544cc28-bd3c-442e-b9c4-c21b9d745107   
1     58797953-d4fb-417a-9e8e-98fa61f0192b   
2     5c265650-4367-4215-8b15-8151c7f6597e   
3     9979e492-0e1b-4348-b3bc-8af5efaaf6f6   
4     f2a025d8-4be5-41bb-8cfa-132052abebeb   
...                                    ...   
2033  0c86a9ba-8b33-49ee-b599-0152e629b5f6   
2034  4379e64d-e6a7-4465-8c9b-54cd7254a6a4   
2035  918da29f-0893-41c7-98e0-a3b227afb773   
2036  048e519c-4e29-43a6-ae85-77d8c8465ba3   
2037  4253ccf7-bae4-4c2a-b907-5d77d91c1a22   

                                                     id               name  \
0     apip-eu-prd-api-aggregatorapi-02.d9ce6d22.eu-c...             ACTIVE   
1     apip-eu-prd-api-aggregatorapi-01.5b220287.eu-c...             ACTIVE   
2     kwontest-02-9720-m00-8892f87b-xnbh6.eu-central...             ACTIVE   
3     kwontest-02-9720-w00-e90d733e-9pgkn.eu-central...             ACTIVE   
4     apip-stg-api-aggregatorapi-01.fb655c2

In [13]:
# for style in document.styles:
#     print(style.name)

document.add_heading('VM 현황', level=1) 
total = len(df_today)

document.add_paragraph(f'총 {total}건', style='List Bullet')

In [16]:
# Pivoting
df = preprocess_df(df_today)
# pivot, total = getPivotTable(df, month)

# Chart - Pie 차트
region_pivot, _ = getPivotTable_new(df, 'Region')
tenant_pivot, _ = getPivotTable_new(df, 'Tenant')

source1 = region_pivot
source2 = tenant_pivot

print(source1)
print(source2)

# Chart
# if incompleted == 0:
chart1 = getPieChart_region(source1)
chart2 = getPieChart_tenant(source2)
# else:
#     source = flatten_2d(data)
#     chart = getStackedHBarChart(source)    

# source = pivot
# if incompleted == 0:
#     chart = getPieChart(source)
# else:
#     chart = getStackedHBarChart1(source)
chart1.save(f'./charts/vm_1.png')
chart2.save(f'./charts/vm_2.png')

table = document.add_table(rows=1, cols=2)

# ----------- 좌측 셀 (리전별) ------------
cell = table.rows[0].cells[0]
paragraph = cell.paragraphs[0]
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 수평 중앙 정렬
run = paragraph.add_run("리전별")
run.font.size = Pt(12)  # 폰트 크기 조절 (선택)
run.bold = True  # 굵게 (선택)

# 이미지 추가 (별도 문단 생성하여 아래로 정렬되게)
paragraph_img = cell.add_paragraph()
paragraph_img.alignment = WD_ALIGN_PARAGRAPH.CENTER
run_img = paragraph_img.add_run()
run_img.add_picture('./charts/vm_1.png', width=Inches(3.3))

# ----------- 우측 셀 (테넌트별) ------------
cell = table.rows[0].cells[1]
paragraph = cell.paragraphs[0]
paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
run = paragraph.add_run("테넌트별")
run.font.size = Pt(12)
run.bold = True

paragraph_img = cell.add_paragraph()
paragraph_img.alignment = WD_ALIGN_PARAGRAPH.CENTER
run_img = paragraph_img.add_run()
run_img.add_picture('./charts/vm_2.png', width=Inches(3.3))

# table = document.add_table(rows=1, cols=2)
# # --- get the first cell of the first row ---
# cell = table.rows[0].cells[0]
# # --- by default a cell has one paragraph with zero runs ---
# paragraph = cell.paragraphs[0]
# # --- add a run in which to place the picture ---
# run = paragraph.add_run()
# # --- add the picture to that run ---
# run.add_text("{0:^42}리전별".format(''))
# run.add_picture(f'./charts/vm_1.png', width=Inches(3.3))
# cell = table.rows[0].cells[1]
# # --- by default a cell has one paragraph with zero runs ---
# paragraph = cell.paragraphs[0]
# # --- add a run in which to place the picture ---
# run = paragraph.add_run()
# # --- add the picture to that run ---
# run.add_text("{0:^42}테넌트별".format(''))
# run.add_picture(f'./charts/vm_2.png', width=Inches(3.3))

    Region    합계    비중
0      FR2  1094  53.5
1      FR7   948  46.3
2  Unknown     4   0.2
    Tenant    합계    비중
0      PRD  1131  55.3
1      STG   911  44.5
2  Unknown     4   0.2


In [17]:
# document.add_paragraph(f'월별 추세', style='List Bullet')
# document.add_picture(f'./charts/cpm_{regions}_{y}_{m}_{d}_2.png')

# Document
document.add_paragraph(f'전일 대비 추가된 VM({len(df_added)}건)', style='List Bullet')
# A-B (삭제된 건)
# df_result_sub = pd.concat([df_premonth,df_curmonth,df_curmonth]).drop_duplicates(keep=False)
# print(df_result_sub)

if len(df_added) == 0:
    pass
else:
    addTable3(df_added)
    
p = document.add_paragraph('')
run = p.add_run()
run.add_break(WD_BREAK.LINE)

document.add_paragraph(f'전일 대비 삭제된 VM({len(df_deleted)}건)', style='List Bullet')
# A-B (삭제된 건)
# df_result_sub = pd.concat([df_premonth,df_curmonth,df_curmonth]).drop_duplicates(keep=False)
# print(df_result_sub)

if len(df_deleted) == 0:
    pass
else:
    addTable3(df_deleted)

document.add_page_break()

In [18]:
document.save(f'C:\\Beomjun\\csv\\vm_report_{current_date}.docx')

## 스토리지 용량 수집(Cinder+Manila)

In [19]:
import time
from datetime import datetime, timedelta
from novaclient import client
import pandas as pd

import os
from keystoneauth1 import session
from keystoneauth1.identity import v3
from odf.opendocument import OpenDocumentSpreadsheet
from odf.table import Table, TableRow, TableCell
from odf.text import P
from cinderclient import client as cinder_client
from manilaclient import client as manila_client

In [20]:
def create_ods_file(file_path, sheet_data_list):
    print("ODS 파일 생성 시작...")

    if os.path.exists(file_path):
        os.remove(file_path)

    doc = OpenDocumentSpreadsheet()

    for sheet_name, header, data_rows in sheet_data_list:
        print(f"📄 시트 생성: {sheet_name}")
        table = Table(name=sheet_name)

        # Header
        header_row = TableRow()
        for col in header:
            cell = TableCell(valuetype="string")
            cell.addElement(P(text=str(col)))
            header_row.addElement(cell)
        table.addElement(header_row)

        # Data rows
        for row in data_rows:
            row_element = TableRow()
            for item in row:
                cell = TableCell(valuetype="string")
                cell.addElement(P(text=str(item if item is not None else "")))
                row_element.addElement(cell)
            table.addElement(row_element)

        doc.spreadsheet.addElement(table)

    # Save file
    with open(file_path, "wb") as f:
        doc.write(f)
        f.flush()

    print(f"✅ ODS 저장 완료: {file_path}")

# 파일명 생성 함수
def get_file_and_topic_names(current_date):
    file_name = f'C:\\Beomjun\\csv\\Volume\\volume_info_{current_date}.ods'
    return file_name

In [21]:
def extract_cinder_data(session):
    cinder = cinder_client.Client('3', session=session)
    volumes = cinder.volumes.list(search_opts={'all_tenants': 1})

    header = ["tenant", "id", "name", "size", "status", "volume_type", "created_at", "availability_zone"]
    data = []

    for v in volumes:
        volume_type = v.volume_type or ""
        tenant = "PRD" if "prd" in volume_type.lower() else "STG" if "stg" in volume_type.lower() else ""

        row = [
            tenant,
            v.id,
            v.name,
            v.size,
            v.status,
            volume_type,
            v.created_at,
            v.availability_zone,
        ]
        data.append(row)

    return ("Cinder_Volumes", header, data)

def extract_manila_data(session):
    manila = manila_client.Client('2', session=session)
    shares = manila.shares.list(search_opts={'all_tenants': 1})

    header = ["tenant", "id", "name", "size", "status", "volume_type", "created_at", "share_proto", "export_location"]
    data = []

    for s in shares:
        volume_type = s.volume_type or ""
        tenant = "PRD" if "prd" in volume_type.lower() else "STG" if "stg" in volume_type.lower() else ""

        row = [
            tenant,
            s.id,
            s.name,
            s.size,
            s.status,
            s.volume_type,
            s.created_at,
            s.share_proto,
            s.export_location
        ]
        data.append(row)

    return ("Manila_Shares", header, data)


In [22]:
def nova_extract():
    current_date = datetime.now().strftime("%Y%m%d")

    auth = v3.Password(
        auth_url='http://cloud-control-vip.eu-central.openstack.h53:5000/v3',
        username='admin',
        password='TldhTl1!',
        user_domain_name='Default',
        project_name='admin',
        project_domain_name='Default'
    )

    sess = session.Session(auth=auth)
    nova = client.Client('2.1', session=sess)

    ods_file_path = get_file_and_topic_names(current_date)
    print("ODS 파일명:", ods_file_path)

    # Cinder & Manila
    cinder_sheet = extract_cinder_data(sess)
    manila_sheet = extract_manila_data(sess)

    # #----------------------------------------------------
    # # 키값 확인 
    # cinder = cinder_client.Client('3', session=sess)
    # volumes = cinder.volumes.list(search_opts={'all_tenants': 1})
    # # 예: 첫 번째 볼륨의 모든 키 확인
    # if volumes:
    #     v = volumes[0]  # 리스트에서 첫 번째 볼륨 선택
    #     volume_dict = v.to_dict()
    
    #     print("🔍 Cinder Volume 모든 필드:")
    #     for key in volume_dict:
    #         print(f"{key}: {volume_dict[key]}")
    # else:
    #     print("❗ 볼륨이 없습니다.")

    # manila = manila_client.Client('2', session=sess)
    # shares = manila.shares.list(search_opts={'all_tenants': 1})
    # if shares:
    #     v = shares[0]  # 리스트에서 첫 번째 볼륨 선택
    #     share_dict = v.to_dict()
    
    #     print("🔍 share Volume 모든 필드:")
    #     for key in share_dict:
    #         print(f"{key}: {share_dict[key]}")
    # else:
    #     print("❗ 볼륨이 없습니다.")
    # #-----------------------------------------------------
    
    # 시트 데이터 리스트 구성
    sheets = [cinder_sheet, manila_sheet]
    create_ods_file(ods_file_path, sheets)

In [23]:
nova_extract()

ODS 파일명: C:\Beomjun\csv\Volume\volume_info_20250526.ods
ODS 파일 생성 시작...
📄 시트 생성: Cinder_Volumes
📄 시트 생성: Manila_Shares
✅ ODS 저장 완료: C:\Beomjun\csv\Volume\volume_info_20250526.ods


## 스토리지 용량 수집(캡처)

In [24]:
from time import sleep
from selenium import webdriver
from threading import Event
from selenium.common.exceptions import NoSuchElementException
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import subprocess
import platform
import certifi
import threading
import sys
import urllib3
import zipfile

screenshots_path = 'ScreenShots'
isExist = os.path.exists(screenshots_path)

if not isExist:
    os.makedirs(screenshots_path)
    print("The new directory is created!")

# Chrome 버전 변경 시, Chrome webdriver 설치
# https://googlechromelabs.github.io/chrome-for-testing/#stable

import winreg
import requests

def get_chrome_version_from_registry():
    reg_paths = [
        r"SOFTWARE\Google\Chrome\BLBeacon",  # 일반 설치
        r"SOFTWARE\WOW6432Node\Google\Chrome\BLBeacon"  # 32bit 설치 경로
    ]

    for reg_path in reg_paths:
        try:
            reg_key = winreg.OpenKey(winreg.HKEY_CURRENT_USER, reg_path)
            version, _ = winreg.QueryValueEx(reg_key, "version")
            winreg.CloseKey(reg_key)
            return version
        except FileNotFoundError:
            continue

    print("[ERROR] Chrome version not found in registry.")
    return None

In [25]:
# 📌 2. 다운로드 및 설치
def download_and_install_chromedriver(chrome_version):
    major_version = chrome_version.split('.')[0]
    # base_url = f"https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/{chrome_version}/win64/chromedriver-win64.zip"
    base_url = f"https://storage.googleapis.com/chrome-for-testing-public/{chrome_version}/win64/chromedriver-win64.zip"

    print(f"[INFO] Downloading ChromeDriver for version {chrome_version}...")
    r = requests.get(base_url, stream=True)
    if r.status_code != 200:
        print(f"[ERROR] Failed to download chromedriver from {base_url}")
        sys.exit(1)

    zip_path = 'chromedriver.zip'
    with open(zip_path, 'wb') as f:
        f.write(r.content)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()

    # chromedriver 위치로 이동
    shutil.move("chromedriver-win64/chromedriver.exe", "./chromedriver.exe")
    shutil.rmtree("chromedriver-win64")
    os.remove(zip_path)
    print("[INFO] ChromeDriver installed successfully.")

chrome_version = get_chrome_version_from_registry()
print(f"[INFO] Detected Chrome version: {chrome_version}")
download_and_install_chromedriver(chrome_version)

[INFO] Detected Chrome version: 137.0.7151.41
[INFO] Downloading ChromeDriver for version 137.0.7151.41...
[INFO] ChromeDriver installed successfully.


In [26]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
exit_event = Event()
os.environ['WDM_SSL_VERIFY'] = '0'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.implicitly_wait(60)

In [27]:
# 웹드라이버 클래스 (생성자/소멸자)
class Driver:
    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless=new")  # 최신 headless 모드
        options.add_argument("--disable-gpu")  # 필요 시 제거
        options.add_argument("--enable-unsafe-swiftshader")
        options.add_argument("--use-gl=swiftshader")
        # options.add_argument("--window-size=1920,1080")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-infobars")
        options.add_argument("--allow-running-insecure-content")
        options.add_argument("--ignore-certificate-errors")
        options.add_argument("--disable-extensions")

        try:
            # 🔥 핵심: ChromeDriverManager로 설치하고 service로 넘김
            # service = Service(ChromeDriverManager().install())
            service = Service(executable_path="C:/Beomjun/chromedriver.exe")
            self.driver = webdriver.Chrome(service=service, options=options)
        except Exception as e:
            print("❌ WebDriver 생성 실패:", e)
            raise
        # service = Service(executable_path='chromedriver')  # 또는 전체 경로
        # self.driver = webdriver.Chrome(service=service, options=options)
        # self.driver = webdriver.Chrome(executable_path='chromedriver', options=options)
        self.driver.set_window_size(1920,1080) 
        sleep(10)
        self.driver.maximize_window()

    def __del__(self):
        try:   
            self.driver.quit() # clean up driver when we are cleaned up
        except Exception:
            pass

thread_local = threading.local()

In [28]:
# 웹페이지 로그인 제공
def login(driver, url, userid, userid_xpath, passwd, passwd_xpath, login_xpath):
    try:
        # Grafana 로그인 페이지로 이동
        driver.get(url)
        driver.switch_to.default_content()
        driver.switch_to.parent_frame()

        try:
            # 고급 버튼 클릭
            driver.find_element('xpath', '//*[@id="details-button"]').click()

            # 이동 버튼 클릭
            driver.find_element('xpath', '//*[@id="proceed-link"]').click()

            # Hcloud 추가
            driver.find_element('xpath', '//*[@id="reactRoot"]/div/main/div[3]/div/div[2]/div/div[2]/div[2]/a').click() 

        except Exception as e:
            print(e)

     
        username = driver.find_element('xpath', userid_xpath)    
        print("id입력 성공"+url)    
        username.clear()
        username.send_keys(userid)
        sleep(3)
        
        password = driver.find_element('xpath', passwd_xpath)
        print("pw입력 성공"+url) 
        password.clear()
        password.send_keys(passwd)
        sleep(3)

        # Login 버튼 클릭
        driver.find_element('xpath', login_xpath).click()
        print("login시도중---->>"+url)
        sleep(3)
   
    except Exception as e:
        return None
        
    return driver


# 로그인 정보 딕셔너리
login_info = {
    "01_EU_Hcloud_Storage": {
        "url": "https://hubble-euce.platform.hcloud.io/grafana/login/generic_oauth",
        # "url": "https://sso.hcloud.hmc.co.kr/auth/realms/hcloud/protocol/openid-connect/auth?client_id=iam-client&redirect_uri=http%3A%2F%2Fhubble-euce.platform.hcloud.io%2Fgrafana%2Flogin%2Fgeneric_oauth&response_type=code&scope=openid+email+profile&state=6kBGz08USQFE2sxJSKZl9LSlao6N9aQCqaBfpIM03cs%3D",
        "userid": "cocop",
        "passwd": "cocop",
        "userid_xpath": '//*[@id="username"]',
        "passwd_xpath": '//*[@id="password"]',
        "login_xpath": '//*[@id="kc-form-login"]/button'
    },
}

def create_driver(bot):
    the_driver = getattr(thread_local, 'the_driver', None)
    print(the_driver)
    # if the_driver is None:
    try:
        the_driver = Driver()
        setattr(thread_local, 'the_driver', the_driver)
        print("새드라이버 생성중~")
    except Exception as e:
        print(e)
        return None

    driver = the_driver.driver

    print("-----create_driver="+bot+"------")
    sleep(3)

    try:
        login_data = login_info.get(bot)
        if login_data:
            url = login_data["url"]
            userid = login_data["userid"]
            passwd = login_data["passwd"]
            userid_xpath = login_data["userid_xpath"]
            passwd_xpath = login_data["passwd_xpath"]
            login_xpath = login_data["login_xpath"]
            driver=login(driver, url, userid, userid_xpath, passwd, passwd_xpath, login_xpath)
            if driver is None: 
                return None
    except Exception as e:
        print(e)

    return driver


def capture_screen(bot):
    
    try:
        driver = create_driver(bot)
        
        if driver is None:
            print("드라이버 생성 실패")
            return  # 드라이버 생성 실패 시 함수 종료
        print("----"+bot+"캡처함수시작---")
        print(f"Capturing the screens started at {datetime.now()}")
        
        start_time = time.time()
        # 최종(대시보드) 페이지 및 저장파일 이름 설정
        urls = {
            "01_EU_Hcloud_Storage": "https://hubble-euce.platform.hcloud.io/grafana/d/dongheon-euce/netapp-euce-summary?orgId=41&viewPanel=2&from=now-3h&to=now",
        }
        filename = f'{screenshots_path}/{bot}_{datetime.now().strftime("%Y%m%d_%H%M")}.png'
        url = urls.get(bot)
        print("url:",url)
        print("bot:",bot)

        if url: 
            if bot == '01_EU_Hcloud_Storage':
                driver.get(url)
                # driver.implicitly_wait(30)
                sleep(30)
                
                WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="mega-menu-toggle"]'))
                ).click()
                driver.execute_script("document.querySelector('#mega-menu-toggle').click();")
                sleep(5)
                driver.set_window_size(1920, 1300)
                # driver.maximize_window()

                # sleep(30)
                # driver.execute_script("document.body.style.zoom=0.75")
                # element = driver.find_element('xpath', '//*[@id="mega-menu-toggle"]')
                # if element.is_displayed():
                #     element.click()
                #     print("click success!")
                # else:
                #     print("Element is not visible in headless mode")


                # driver.set_window_size(1920, 1500)
                # driver.maximize_window()
                # sleep(10)
                # get total page dimensions
                # total_width = driver.execute_script("return document.body.scrollWidth")
                # total_height = driver.execute_script("return document.body.scrollHeight")

                # set to full size
                # driver.set_window_size(total_width, total_height)
                # sleep(2)

                # zoom out
                sleep(5)
                # driver.execute_script("document.body.style.zoom='0.50'")
                # sleep(2)
                # driver.execute_script("document.body.style.zoom='0.75'")
                # sleep(10)
                driver.save_screenshot(filename)    
                driver.close()
                driver.quit()
                print('스토리지 캡쳐 완료!')

            else:
                
                driver.get(url)
                sleep(30)
                driver.set_window_size(1920, 1080)
                driver.maximize_window()
               
                sleep(50)
                driver.execute_script("document.body.style.zoom=0.75")
                sleep(30)
                driver.save_screenshot(filename)
                driver.close()
                driver.quit()
        else: 
            print("Error occurred while capture!!")
        print("----"+bot+"캡처함수종료-----")
    # 예외 처리
    except Exception as e:
        print(f"Error occurred: {e}")
        print("\n... Program Stopped Manually!")
        return

In [29]:
def main_capture():

    number_threads = 1
    
    bots = [
        '01_EU_Hcloud_Storage',
    ]

    with ThreadPoolExecutor(max_workers=number_threads) as pool:
        try:
            pool.map(capture_screen, bots)
        except KeyboardInterrupt:
            print('Caught keyboardinterrupt')
            pass

if __name__ == "__main__":
    try:
        main_capture()
    except KeyboardInterrupt:
        print('Caught keyboardinterrupt')
        pass

    import gc
    gc.collect() # a little extra insurance
    print("---------------------------end----------------------------")

None
새드라이버 생성중~
-----create_driver=01_EU_Hcloud_Storage------
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="details-button"]"}
  (Session info: chrome=137.0.7151.41); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff628179235+2853861]
	GetHandleVerifier [0x0x7ff627ed3af0+79008]
	(No symbol) [0x0x7ff627c99bda]
	(No symbol) [0x0x7ff627cf00f6]
	(No symbol) [0x0x7ff627cf03ac]
	(No symbol) [0x0x7ff627d43b07]
	(No symbol) [0x0x7ff627d184ff]
	(No symbol) [0x0x7ff627d408f5]
	(No symbol) [0x0x7ff627d18293]
	(No symbol) [0x0x7ff627ce1061]
	(No symbol) [0x0x7ff627ce1df3]
	GetHandleVerifier [0x0x7ff6281a414d+3029757]
	GetHandleVerifier [0x0x7ff62819e56d+3006237]
	GetHandleVerifier [0x0x7ff6281bd5f2+3133346]
	GetHandleVerifier [0x0x7ff627eed9be+185198]
	GetHandleVerifier [0x0x7ff627ef4f0f+215231]
	GetHandleVerifier [0x0x7

## Proxmox 수집

In [30]:
from proxmoxer import ProxmoxAPI
from datetime import datetime, timedelta
import pandas as pd
import re

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 여러 Proxmox 서버 연결 정보 리스트
proxmox_servers = [
    {'host': 'proxmox-eu-central-1.hcloud.io:8006', 'user': 'root@pam', 'password': 'EUccs2!@#$'},
    {'host': 'proxmox-lms-prd-eu-central-1.hcloud.io:8006', 'user': 'root@pam', 'password': 'EUccs2!@#$'},
    {'host': 'proxmox-lms-stg-eu-central-1.hcloud.io:8006', 'user': 'root@pam', 'password': 'EUccs2!@#$'},
    {'host': 'proxmox-eu-central-2.hcloud.io:8006', 'user': 'root@pam', 'password': 'EUccs2!@#$'},
    {'host': 'proxmox-lms-prd-eu-central-2.hcloud.io:8006', 'user': 'root@pam', 'password': 'EUccs2!@#$'},
    {'host': 'proxmox-lms-stg-eu-central-2.hcloud.io:8006', 'user': 'root@pam', 'password': 'EUccs2!@#$'}
]

In [31]:
# uptime 변환 함수
def format_uptime(seconds):
    days = seconds // 86400
    hours = (seconds % 86400) // 3600
    return f"{days}d {hours}h"

current_date = datetime.now().strftime("%Y%m%d")

# Sheet 이름 변환 함수
def format_sheet_name(hostname):
    hostname = hostname.replace(':', '_')
    hostname = re.sub(r'eu-central-1\.hcloud\.io', 'FR2', hostname)
    hostname = re.sub(r'eu-central-2\.hcloud\.io', 'FR7', hostname)
    return hostname

In [32]:
# Excel writer 생성
with pd.ExcelWriter(f"C:\\Beomjun\\csv\\Proxmox\\proxmox_info_{current_date}.xlsx") as writer:
    for server in proxmox_servers:
        proxmox = ProxmoxAPI(server['host'], user=server['user'], password=server['password'], verify_ssl=False)

        cluster_resources = proxmox.cluster.resources.get(type='vm')
        cluster_df = pd.DataFrame(cluster_resources)

        vm_ip_list = []

        for node in proxmox.nodes.get():
            node_name = node['node']
            vms = proxmox.nodes(node_name).qemu.get()

            for vm in vms:
                vmid = vm['vmid']
                config = proxmox.nodes(node_name).qemu(vmid).config.get()
                ipconfig = config.get('ipconfig0', None)

                ip = None
                if ipconfig:
                    match = re.search(r'ip=([\d.]+)', ipconfig)
                    if match:
                        ip = match.group(1)

                vm_ip_list.append({
                    'vmid': int(vmid),
                    'cloud_init_ip': ip
                })

        ip_df = pd.DataFrame(vm_ip_list)

        merged_df = pd.merge(cluster_df, ip_df, on='vmid', how='left')

        merged_df['cpu'] = (merged_df['cpu'] * 100).round(2)
        merged_df['mem'] = (merged_df['mem'] / (1024**3)).round(2)
        merged_df['maxmem'] = (merged_df['maxmem'] / (1024**3)).round(2)
        merged_df['maxdisk'] = (merged_df['maxdisk'] / (1024**3)).round(2)

        merged_df['uptime'] = merged_df['uptime'].apply(format_uptime)

        final_df = merged_df[['vmid', 'name', 'node', 'status', 'cloud_init_ip', 
                              'cpu', 'maxcpu', 'mem', 'maxmem', 'disk', 'maxdisk', 'uptime']]

        # 각 서버별로 Excel Sheet 출력
        sheet_name = format_sheet_name(server['host'])
        final_df.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"Data from {server['host']} exported to sheet '{sheet_name}'")

Data from proxmox-eu-central-1.hcloud.io:8006 exported to sheet 'proxmox-FR2_8006'
Data from proxmox-lms-prd-eu-central-1.hcloud.io:8006 exported to sheet 'proxmox-lms-prd-FR2_8006'
Data from proxmox-lms-stg-eu-central-1.hcloud.io:8006 exported to sheet 'proxmox-lms-stg-FR2_8006'
Data from proxmox-eu-central-2.hcloud.io:8006 exported to sheet 'proxmox-FR7_8006'
Data from proxmox-lms-prd-eu-central-2.hcloud.io:8006 exported to sheet 'proxmox-lms-prd-FR7_8006'
Data from proxmox-lms-stg-eu-central-2.hcloud.io:8006 exported to sheet 'proxmox-lms-stg-FR7_8006'


## LB VIP 수량 변화 수집

In [33]:
import requests
import os
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta
import urllib3
import subprocess
from openpyxl import Workbook

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# .env 로드
load_dotenv(dotenv_path="config/ctrix.env")

# 환경변수 로드
# IP 리스트 만들기
NS_IPS_FR2 = os.getenv("NS_IPS_FR2", "").split(",")
NS_IPS_FR7 = os.getenv("NS_IPS_FR7", "").split(",")

# 각각 공백 제거
ip_list_fr2 = [ip.strip() for ip in NS_IPS_FR2 if ip.strip()]
ip_list_fr7 = [ip.strip() for ip in NS_IPS_FR7 if ip.strip()]

NITRO_USERNAME = os.getenv("NITRO_USERNAME")
NITRO_PASSWORD = os.getenv("NITRO_PASSWORD")
HEADERS = {"Content-Type": "application/json"}
VERIFY_SSL = False  # 자체서명 인증서 무시

VPN_FR2_NAME = "EU2-FR2"
VPN_FR7_NAME = "EU2-FR7"
USERNAME = "spark"
PASSWORD = "tmvkzm1!"

# IP와 Tenant 매핑 로딩
IP_TENANT_MAPPING = {}
tenant_mapping_fr2 = os.getenv("TENANT_MAPPING_FR2", "")
tenant_mapping_fr7 = os.getenv("TENANT_MAPPING_FR7", "")

for item in tenant_mapping_fr2.split(","):
    if ":" in item:
        ip, tenant = item.split(":")
        IP_TENANT_MAPPING[ip.strip()] = tenant.strip()

for item in tenant_mapping_fr7.split(","):
    if ":" in item:
        ip, tenant = item.split(":")
        IP_TENANT_MAPPING[ip.strip()] = tenant.strip()

print("mapping info!", IP_TENANT_MAPPING)

mapping info! {'172.28.7.8': 'FR2_Admin', '172.28.7.14': 'FR2_PRD-A', '172.28.7.20': 'FR2_PRD-B', '172.28.7.26': 'FR2_PRD-C', '172.28.7.32': 'FR2_PRD-D', '172.28.7.11': 'FR2_STG-A', '172.28.7.17': 'FR2_STG-B', '172.28.7.23': 'FR2_STG-C', '172.28.7.29': 'FR2_STG-D', '172.30.7.8': 'FR7_Admin', '172.30.7.14': 'FR7_PRD-A', '172.30.7.20': 'FR7_PRD-B', '172.30.7.26': 'FR7_PRD-C', '172.30.7.32': 'FR7_PRD-D', '172.30.7.11': 'FR7_STG-A', '172.30.7.17': 'FR7_STG-B', '172.30.7.23': 'FR7_STG-C', '172.30.7.29': 'FR7_STG-D'}


In [34]:
# VPN 관련 함수
def is_vpn_connected(vpn_name):
    result = subprocess.run(['rasdial'], capture_output=True, text=True)
    return vpn_name.lower() in result.stdout.lower()

def connect_vpn(vpn_name, username, password):
    print(f"[INFO] Connecting to VPN: {vpn_name}")
    subprocess.run(['rasdial', vpn_name, username, password])

def disconnect_vpn(vpn_name):
    print(f"[INFO] Disconnecting VPN: {vpn_name}")
    subprocess.run(['rasdial', vpn_name, '/disconnect'])

# Citrix API 함수
def login(session, base_url):
    url = f"{base_url}/config/login"
    payload = {"login": {"username": NITRO_USERNAME, "password": NITRO_PASSWORD}}
    response = session.post(url, json=payload, headers=HEADERS)
    response.raise_for_status()

def get_lb_vips(session, base_url):
    url = f"{base_url}/config/lbvserver"
    response = session.get(url, headers=HEADERS)
    response.raise_for_status()
    lb_list = response.json().get("lbvserver", [])
    return lb_list

def logout(session, base_url):
    url = f"{base_url}/config/logout"
    payload = {"logout": {}}
    session.post(url, json=payload, headers=HEADERS)

In [35]:
# LB 수집 함수
def collect_lb_info(region, vpn_name, ip_list):
    if is_vpn_connected(vpn_name):
        print(f"[INFO] VPN {vpn_name} is already connected.")
    else:
        print(f"[INFO] VPN {vpn_name} not connected. Attempting connection...")
        connect_vpn(vpn_name, USERNAME, PASSWORD)

    lb_data = []
    for ip in ip_list:
        tenant = IP_TENANT_MAPPING.get(ip, "UNKNOWN")
        print(f"[INFO] Collecting LB info from {tenant} ({ip})")
        base_url = f"https://{ip}/nitro/v1"
        session = requests.Session()
        session.verify = VERIFY_SSL

        try:
            login(session, base_url)
            lb_list = get_lb_vips(session, base_url)

            for lb in lb_list:
                lb_name = lb.get('name')
                lb_ip = lb.get('ipv46')
                lb_port = lb.get('port')
                service_type = lb.get('servicetype')
                
                if lb_ip == "0.0.0.0":
                    continue
                
                lb_data.append([tenant, ip, lb_name, lb_ip, lb_port, service_type])

        except requests.RequestException as e:
            print(f"❌ 오류 발생 ({ip}):", e)

        finally:
            logout(session, base_url)

    disconnect_vpn(vpn_name)
    
    return lb_data  # ✅ DataFrame이 아닌 리스트 반환


In [36]:
def compare_yesterday():
    # 날짜 설정
    current_date = datetime.now().strftime("%Y%m%d")
    yesterday_date = (datetime.now() - timedelta(days=1)).strftime("%Y%m%d")
    
    print("today:", current_date)
    print("yesterday:", yesterday_date)
    
    # 경로 설정
    today_path = f"C:\\Beomjun\\csv\\LB\\lb_info_{current_date}.ods"
    yesterday_path = f"C:\\Beomjun\\csv\\LB\\lb_info_{yesterday_date}.ods"
    
    # file_path=os.path.abspath("")
    # srcpath = findfile(f"eu_adminrc_{yesterday_date}.csv.org", file_path)
    # dir, file = os.path.split(srcpath)
    # shutil.copy2(srcpath, dir+f"\eu_adminrc_{yesterday_date}.csv")
        
    # CSV 로드
    df_today = pd.read_excel(today_path, engine='odf')
    df_yesterday = pd.read_excel(yesterday_path, engine='odf')
    
    # ID 기준으로 차집합 연산
    today_ids = set(df_today['LB_Name'])
    yesterday_ids = set(df_yesterday['LB_Name'])
    
    # 추가된 VM: 오늘에는 있고 어제는 없던 것
    added_ids = today_ids - yesterday_ids
    df_added = df_today[df_today['LB_Name'].isin(added_ids)]
    
    # 삭제된 VM: 어제는 있었는데 오늘은 없는 것
    deleted_ids = yesterday_ids - today_ids
    df_deleted = df_yesterday[df_yesterday['LB_Name'].isin(deleted_ids)]
    
    # 결과 출력
    print("\n✅ 오늘 추가된 VM:")
    print(df_added)
    
    print("\n❌ 오늘 삭제된 VM:")
    print(df_deleted)

    return df_added, df_deleted


In [37]:
def add_document(df_today, df_added, df_deleted):
    document.add_heading('LB 현황', level=1) 
    total = len(df_today)
    
    document.add_paragraph(f'총 {total}건', style='List Bullet')

    # Pivoting
    df = preprocess_df_LB(df_today)
    # pivot, total = getPivotTable(df, month)
    
    # Chart - Pie 차트
    region_pivot, _ = getPivotTable_new(df, 'Tenant')
    # tenant_pivot, _ = getPivotTable_new(df, 'Tenant')
    
    source1 = region_pivot
    # source2 = tenant_pivot
    
    print(source1)
    # print(source2)
    
    # Chart
    # if incompleted == 0:
    chart1 = getPieChart_tenant(source1)
    # chart2 = getPieChart_tenant(source2)
    # else:
    #     source = flatten_2d(data)
    #     chart = getStackedHBarChart(source)    
    
    # source = pivot
    # if incompleted == 0:
    #     chart = getPieChart(source)
    # else:
    #     chart = getStackedHBarChart1(source)
    chart1.save(f'./charts/LB_1.png')
    # chart2.save(f'./charts/vm_2.png')

    # p = document.add_paragraph('')
    # run = p.add_run()
    # run.add_break(WD_BREAK.LINE)
    
    table = document.add_table(rows=1, cols=1)
    cell = table.rows[0].cells[0]
    paragraph = cell.paragraphs[0]
    paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER  # 수평 중앙 정렬
    run = paragraph.add_run("테넌트별")
    run.font.size = Pt(12)  # 폰트 크기 조절 (선택)
    run.bold = True  # 굵게 (선택)
    
    # 이미지 추가 (별도 문단 생성하여 아래로 정렬되게)
    paragraph_img = cell.add_paragraph()
    paragraph_img.alignment = WD_ALIGN_PARAGRAPH.CENTER
    run_img = paragraph_img.add_run()
    run.add_picture(f'./charts/LB_1.png')
    # run.add_picture(f'./charts/LB_1.png', width=Inches(3.3))
    
    # Document
    document.add_paragraph(f'전일 대비 추가된 LB({len(df_added)}건)', style='List Bullet')
    
    if len(df_added) == 0:
        pass
    else:
        addTable_LB(df_added)
        
    p = document.add_paragraph('')
    run = p.add_run()
    run.add_break(WD_BREAK.LINE)
    
    document.add_paragraph(f'전일 대비 삭제된 LB({len(df_deleted)}건)', style='List Bullet')
    # A-B (삭제된 건)
    # df_result_sub = pd.concat([df_premonth,df_curmonth,df_curmonth]).drop_duplicates(keep=False)
    # print(df_result_sub)
    
    if len(df_deleted) == 0:
        pass
    else:
        addTable_LB(df_deleted)

    document.save(f'C:\\Beomjun\\csv\\total_report_{current_date}.docx')

In [38]:
# 최종 실행
if __name__ == "__main__":
    current_date = datetime.now().strftime("%Y%m%d")
    ods_file = f"C:\\Beomjun\\csv\\LB\\lb_info_{current_date}.ods"

    print("[INFO] Starting collection for FR2")
    fr2_data = collect_lb_info("FR2", VPN_FR2_NAME, ip_list_fr2)

    print("[INFO] Starting collection for FR7")
    fr7_data = collect_lb_info("FR7", VPN_FR7_NAME, ip_list_fr7)

    # ✅ 데이터 병합
    combined_data = fr2_data + fr7_data
    columns = ["Tenant", "NS_IP", "LB_Name", "LB_VIP", "Port", "Service_Type"]
    df_combined = pd.DataFrame(combined_data, columns=columns)

    # # ✅ ODS 파일 저장
    with pd.ExcelWriter(ods_file, engine='odf') as writer:
        df_combined.to_excel(writer, sheet_name="LB_Info", index=False)

    df_added, df_deleted = compare_yesterday()
    add_document(df_combined, df_added, df_deleted)
    
    print(f"\n✅ ODS 파일이 생성되었습니다: {ods_file}")

[INFO] Starting collection for FR2
[INFO] VPN EU2-FR2 is already connected.
[INFO] Collecting LB info from FR2_Admin (172.28.7.8)
[INFO] Collecting LB info from FR2_PRD-A (172.28.7.14)
[INFO] Collecting LB info from FR2_PRD-B (172.28.7.20)
[INFO] Collecting LB info from FR2_PRD-C (172.28.7.26)
[INFO] Collecting LB info from FR2_PRD-D (172.28.7.32)
[INFO] Collecting LB info from FR2_STG-A (172.28.7.11)
[INFO] Collecting LB info from FR2_STG-B (172.28.7.17)
[INFO] Collecting LB info from FR2_STG-C (172.28.7.23)
[INFO] Collecting LB info from FR2_STG-D (172.28.7.29)
[INFO] Disconnecting VPN: EU2-FR2
[INFO] Starting collection for FR7
[INFO] VPN EU2-FR7 not connected. Attempting connection...
[INFO] Connecting to VPN: EU2-FR7
[INFO] Collecting LB info from FR7_Admin (172.30.7.8)
[INFO] Collecting LB info from FR7_PRD-A (172.30.7.14)
[INFO] Collecting LB info from FR7_PRD-B (172.30.7.20)
[INFO] Collecting LB info from FR7_PRD-C (172.30.7.26)
[INFO] Collecting LB info from FR7_PRD-D (172.30.